# Scrape TripAdvisor Reviews

### Extract number of reviews per airline

### Choose top 10 airlines using total reviews

### Scrape top airlines by Region / Star Rating

#### Get top airline per page of results

In [11]:
# Install playwright library

# !pip3 install playwright   # for package

# !pip3 install asyncio      # for async calls

# !playwright install        # for browser binaries

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [130]:
# Import relevant libaries

import os 
import datetime 
import re

# Libraries for lyrics scrape section

from playwright.sync_api import sync_playwright

# Libraries for lyrics scrape section

import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter 
import random

# Additional libraries

import shutil
import pprint
import numpy as np
import pandas as pd

In [ ]:
class SearchPage:
    def __init__(self, page):
        self.page = page
        self.search_term_input = page.locator('[aria-label="Enter your search term"]')

    async def navigate(self):
        await self.page.goto("https://bing.com")

    async def search(self, text):
        await self.search_term_input.fill(text)
        await self.search_term_input.press("Enter")

In [131]:
import asyncio
from playwright.async_api import async_playwright

# Helper to concat list of lists

def flatten_sum(matrix):
    return sum(matrix, [])

# Function to scrape items on page

async def scrape_page(page):
    info = list()
    airline_name = page.locator('//div[@class="airlineName"]')
    airline_review_count = page.locator('//div[@class="airlineReviews"]')
    airline_name = page.locator('//div[@class="airlineSummary"]/a/div[@class="airlineName"]')
    airline_name = page.locator('//div[@class="airlineSummary"]')
    airline_review_count = page.locator('//div[@class="airlineSummary"]/a[@class="detailsLink"]')
    
    names = await airline_name.all_inner_texts()
    reviews = await airline_review_count.all_inner_texts()

    idx = 0
    for idx in range(len(names)):
        name = names[idx].split('\n')[0]
        el = {
            "airline_name": f"{name}",
            "airline_reviews": f"{reviews[idx]}"
        }
        info.append(el)
        idx += 1
    return info

# Function to go to next page

async def paginate(page, browser):
    try:
        next_page = page.locator('//span[@class="nav next ui_button primary"]')
        await next_page.click()
    except:
        await browser.close()

# Function to check for next page element

async def check_next(page):
    try:
        next_page = None
        if (page.locator('//span[@class="nav next ui_button primary"]')):
            next_page = page.locator('//span[@class="nav next ui_button primary"]')
        return next_page
    except:
        next_page = None
        return next_page

# Main execution function using functions above

async def main():
    async with async_playwright() as p:

        # Init browser

        browser = await p.chromium.launch(headless=False, slow_mo=1500)
        page = await browser.new_page()

        # Go to trip advisor

        await page.goto("https://www.tripadvisor.com/Airlines")

        # Scrape page and paginate

        airline_details = list()
        pg = 0
        while pg < 63:
            next_page = await check_next(page)
            scraped = await scrape_page(page)
            airline_details.append(scraped)
            pg += 1
            await paginate(page, browser)

        print("Length before concat: ", len(airline_details)) 
        airline_deets = flatten_sum(airline_details)
        print("Length after concat: ", len(airline_deets))      
        return airline_deets


In [132]:
information = await main()

Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button present...
Next page button pre

In [133]:
import string

# Helper to remove punctuation and clean reviews string

def conv_int(text):
    arr = text.split(' ')
    strg = arr[0].translate(str.maketrans('', '', string.punctuation))
    return strg

In [134]:
# Extract number of reviews per airline
 
airlines_df = pd.DataFrame(information)
airlines_df['review_count'] = airlines_df['airline_reviews'].apply(conv_int)
airlines_df['review_count'] = pd.to_numeric(airlines_df['review_count'])
airlines_df.sort_values('review_count', ascending=False).head(20)

,airline_name,airline_reviews,review_count
557,Transavia,"109,461 reviews",109461.0
454,Ryanair,"87,299 reviews",87299.0
107,American Airlines,"77,584 reviews",77584.0
225,Emirates,"64,105 reviews",64105.0
209,Delta Air Lines,"61,648 reviews",61648.0
219,easyJet,"61,547 reviews",61547.0
160,British Airways,"58,539 reviews",58539.0
575,United Airlines,"54,083 reviews",54083.0
505,Southwest Airlines,"45,324 reviews",45324.0
350,LATAM Airlines,"40,953 reviews",40953.0


# Scrape SkyTrax Ratings